# Submission - Exercises sheet 2
MA-INF 4236 - Advanced Methods for Text Mining, So24<br>
Yara Elwakeel 50135730<br>
Aksa Aksa 50146305<br>
Sadia Naseer 50194450<br>
Ali Ather 50194294<br>
Affan Zafar 50167759<br>
june 19, 2024<br>

## 1)

## 2)

## 3)

## 4) 

## 5) 
### 5.1)
#### MLP 
A Multi-Layer Perceptron (MLP) consists of an input layer, one or more hidden layers, and an output layer. The connections between these layers are feedforward and unidirectional, meaning each neuron in a layer is connected to every neuron in the subsequent layer. This fully connected nature allows MLPs to effectively capture complex patterns in the data. <br>
MLPs are best suited for static datasets, where the data points are independent and identically distributed (i.i.d.), such as image classification, regression tasks, and other problems where the order of the data does not matter. Because MLPs do not have any inherent memory of previous inputs, they treat each input individually without considering any temporal dependencies.
#### RNN 
A Recurrent Neural Network (RNN) can be considered an extension of an MLP with additional features that enable it to handle sequential data. The key feature that distinguishes RNNs from MLPs is the presence of feedback connections. These connections allow the network to transmit information from neurons in later layers back to neurons in earlier layers. This introduces a notion of time delay, ensuring that information from one time step 
𝑡 is carried over to the next time step t+1 without alteration. This mechanism enables the network to maintain a memory of past states and capture sequential dependencies over time.<br>
RNNs are particularly effective for tasks that involve sequential data where the order and timing of the inputs matter. This includes applications such as language modeling, where the meaning of a word can depend on the previous words in the sentence, and time-series prediction
<br>
<br>
an example of vanilla RNN is elman network, at which the system contains a system state(memory / context neuron) $h_t$ to model dependencies at a given time t:
$$
h_t = f^h (Bh_{t-1}+ Ax_t)
$$
$$
x_{t+1} = f^0 (Ch_t)
$$


### 5.2)

long term dependency is a problem most vanilla RNN methods suffers from, which refers to the difficulty they face when learning and capturing dependencies that span over a considerable number of time steps, as by each info passing by each time step, the gradient can either explode or vanish during training. this phenomena makes it challenging for the network to effectively learn and retain info over long dependencies.
LSTM and GRU are explicitly designed to counter this problem where the main idea is control information flow through gating mechanisms
- LSTMs: Use multiple gates to control the cell state explicitly, allowing selective forgetting, updating, and outputting of information. This fine-grained control helps in maintaining long-term dependencies.

- GRUs: Use reset and update gates to merge the hidden state and memory state, simplifying the process while still effectively managing the flow of information.

#### Structures 
##### LSTM 
LSTMs divide the context management problem into two subproblems: removing information no longer
needed from the context, and adding information likely to be needed for later decision making.<br>
the key for LSTM success is cell state which maintains information across time steps thus preserving long term dependencies (long term memory) and gates that regulates the flow of information. which are 3 in total, these gates share a common design pattern; each consists of a feedforward layer, followed by a sigmoid activation function, followed by a pointwise
multiplication with the layer being gated:
- forget gate:
<br>
The purpose of this gate is
to delete information from the cell state that is no longer needed.(fraction of the previous cell states to forget)
$$
f_t = \sigma (Ax_t + Bh_{t-1}+a) 
$$
- input gate <br>
This is done in two stages, first is to compute the actual information we need to extract from the previous hidden state and current inputs, a candidate value that we might want to save into our cell state.
$$
\hat{c_t} = tanh(Cx_t + Dh_{t-1}+ b)
$$
to control the amount of fraction of new learned information same as before (percentage of potential memory to remember)
$$
i_T = \sigma (Ex_t + Fh_{t-1}+c) 
$$
then update the cell state, by multiplying the old state by $f_t$, forgetting the things that we decided to forget and add the new fraction of learned information 
$$
c_t = f_t \odot c_{t-1} + i_t \odot \hat{c_t}
$$
- output gate: 
<br>
The final gate we’ll use, which is used to decide what information is required for the current hidden state 
$$
o_t = \sigma (Gx_t + Hh_{t-1}+d) 
$$
$$
h_t = o_t \odot tanh(c_t)
$$
where $h_t$ is the hidden state, that provides the output for the LSTM at each time step.




#### GRU 
a GRU is a simplified version of LSTM , as instead of three gates(input, output and forget) now GRU has only two gates. Additionally,GRU combines the cell state and hidden state.: 
- update gate :
which determines how much of the past information (previous hidden state $h_{t-1}$)  should be carried forward along the future(current hidden state $h_t$)
<br>
$$
z_t = \sigma(Ax_t +Bh_{t−1} +a)
$$
where $\sigma $ is the sigmoid activation function , A,B are weight matrices and a is the bias term .
- Reset Gate: 
<br>
determines how much of the previous hidden state ($h_{t-1}$) should be discarded. 
$$
r_t = \sigma(Cx_t +Dh_{t−1} +b)
$$
<br>
all that is left is to compute the candidate hidden state , incorporating the reset gate to control how much of the previous hidden state influence the candidate state .

$$ 
\hat{h_t} = tanh (Ex_t + F(r_t \odot h_{t-1}) + c)
$$

afterwords, combine the previous hidden state and the candidate hidden state using the update gate that determines the proportion of the candidate activation and the previous hidden state to use. when $ z_t $ is close to 1 , more of the candidate state is used and vise versa. 
$$
h_t = z_t \odot \hat{h_t} + (1- z_t) \odot h_{t-1}
$$


#### Main differences between LSTM and GRU : 
- LSTM has an extra gate and thus extra parameters than GRU , which gives it more flexibility and expressiveness but more computational cost and risk of overfitting. on the contrary, GRU possess fewer gates and parameters which makes it faster and simpler but less powerful and adaptable 

- LSTM has separate cell state and hidden state, which allows it to store and output different views of the input , while GRU has single hidden state that serves both purposes, which may limit it's capacity  

## 6) 

### 6.1)

In [1]:
# required packages
%pip install -q sentence-transformers datasets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement chromodb (from versions: none)
ERROR: No matching distribution found for chromodb


In [2]:
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import numpy as np

# Load the dataset
dataset = load_dataset('mt0rm0/movie_descriptors_small')

# print(dataset['train'])
# Extract movie titles and synopses
movie_titles = [entry['title'] for entry in dataset['train']]
movie_synopses = [entry['overview'] for entry in dataset['train']]

c:\Users\yarae\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Load the SentenceBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all movie synopses
synopsis_embeddings = model.encode(movie_synopses, convert_to_tensor=True)

c:\Users\yarae\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
def recommend_movies(query, top_k=5):
    # Compute the embedding for the query
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Compute cosine similarities between the query embedding and all synopsis embeddings
    similarities = util.pytorch_cos_sim(query_embedding, synopsis_embeddings)[0]
    
    # Get the indices of the top_k most similar synopses
    top_k_indices = similarities.topk(k=top_k).indices
    
    # return the corresponding movie titles
    recommended_movies = [movie_titles[idx] for idx in top_k_indices]
    return recommended_movies


In [12]:
# Example queries
queries = [
    "What movie is set in ancient Rome (similar to Gladiator)?",
    "Please recommend me some movies that are both romantic and funny.",
    "Which movies are about astronauts and starships?",
    "Looking for a thrilling heist movie.",
    "Can you suggest a movie with a strong female lead?",
    "I want to watch a movie about artificial intelligence.",
    "Recommend me some heartwarming family movies.",
    "What's a good mystery film with lots of twists?"
]

# Test the system
for query in queries:
    print(f"Query: {query}")
    recommendations = recommend_movies(query)
    for idx, movie in enumerate(recommendations):
        print(f"{idx+1}. {movie}")
    print("\n")


Query: What movie is set in ancient Rome (similar to Gladiator)?
1. Agora
2. Gladiator
3. Cleopatra
4. La Dolce Vita
5. The Man Who Knew Too Much


Query: Please recommend me some movies that are both romantic and funny.
1. About Last Night
2. Midnight in Paris
3. Hysteria
4. Closer
5. Last Night


Query: Which movies are about astronauts and starships?
1. Forbidden Planet
2. Galaxy Quest
3. You Only Live Twice
4. Treasure Planet
5. The Right Stuff


Query: Looking for a thrilling heist movie.
1. The Getaway
2. The Collector
3. Drive
4. Baby Driver
5. The Score


Query: Can you suggest a movie with a strong female lead?
1. Swimming Pool
2. To Die For
3. Thelma & Louise
4. The Young Victoria
5. Under the Skin


Query: I want to watch a movie about artificial intelligence.
1. Pi
2. Ex Machina
3. King Kong
4. Ghost in the Shell 2: Innocence
5. Gattaca


Query: Recommend me some heartwarming family movies.
1. Midnight in Paris
2. This Is Where I Leave You
3. Legends of the Fall
4. Scary Mo